# Redes Siamesas

Este notebook introduz o conceito de Redes Siamesas (Siamese Networks), uma arquitetura de rede neural projetada não para classificar entradas, mas para aprender um espaço de características (embedding) onde a distância entre amostras semelhantes é minimizada e a distância entre amostras distintas é maximizada. Utilizaremos o dataset Labeled Faces in the Wild (LFW) para treinar um modelo capaz de verificar se dois retratos faciais pertencem à mesma pessoa. A implementação será realizada em PyTorch, com foco na definição da arquitetura, na função de custo (Contrastive Loss) e no processo de inferência para verificação.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from sklearn.datasets import fetch_lfw_pairs
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

### Carregamento e Preparação do Dataset (LFW)

Utilizaremos o dataset Labeled Faces in the Wild (LFW), especificamente a versão pré-processada em pares disponibilizada pelo `scikit-learn`. Este conjunto de dados já fornece pares de imagens rotulados como "genuínos" (mesma pessoa, label 1) ou "impostores" (pessoas diferentes, label 0). Realizaremos o download e, em seguida, dividiremos o conjunto de treinamento original em um conjunto de treinamento e um conjunto de validação para monitorar o aprendizado.

In [ ]:
lfw = fetch_lfw_pairs(subset='train', resize=0.5, color=False)

pairs_train, pairs_val, labels_train, labels_val = train_test_split(
    lfw.pairs, lfw.target, test_size=0.2, random_state=42, stratify=lfw.target
)

lfw_test = fetch_lfw_pairs(subset='test', resize=0.5, color=False)
pairs_test, labels_test = lfw_test.pairs, lfw_test.target

print(f"Treino: {len(pairs_train)}, Validação: {len(pairs_val)}, Teste: {len(pairs_test)}")

### Visualização dos Pares de Imagens

Para compreender a natureza dos dados, visualizaremos alguns exemplos. Mostraremos um par positivo (duas imagens da mesma pessoa) e um par negativo (imagens de pessoas distintas).

In [ ]:
def show_pair(pair, label):
    fig, axes = plt.subplots(1, 2, figsize=(6, 3))
    
    axes[0].imshow(pair[0], cmap='gray')
    axes[0].set_title('Image 1')
    axes[0].axis('off')
    
    axes[1].imshow(pair[1], cmap='gray')
    axes[1].set_title('Image 2')
    axes[1].axis('off')
    
    title = 'Same Person' if label == 1 else 'Different People'
    fig.suptitle(title, fontsize=14)
    plt.show()

# Encontrar um exemplo de cada
positive_idx = np.where(labels_train == 1)[0][0]
negative_idx = np.where(labels_train == 0)[0][0]

print("Exemplo de Par Positivo (Mesma Pessoa):")
show_pair(pairs_train[positive_idx], labels_train[positive_idx])

print("Exemplo de Par Negativo (Pessoas Diferentes):")
show_pair(pairs_train[negative_idx], labels_train[negative_idx])

### Definição do Dataset PyTorch

Para integrar os dados ao ecossistema PyTorch, criamos uma classe `Dataset` customizada. Esta classe encapsula a lógica de acesso aos pares de imagens e seus respectivos rótulos, além de aplicar as transformações necessárias (conversão para Tensor e normalização).

In [ ]:
class LFWPairedDataset(Dataset):
    def __init__(self, pairs, labels, transform=None):
        self.pairs = pairs
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img1 = np.expand_dims(self.pairs[idx][0], axis=-1)
        img2 = np.expand_dims(self.pairs[idx][1], axis=-1)
        label = torch.tensor(self.labels[idx], dtype=torch.float32)

        if self.transform:
            img1 = self.transform(img1)
            img2 = self.transform(img2)
            
        return img1, img2, label

In [ ]:
transform_aug = T.Compose([
    T.ToPILImage(),
    T.Resize((100, 100)),
    T.RandomHorizontalFlip(),
    T.RandomRotation(10),
    T.ToTensor(),
    T.Normalize((0.5,), (0.5,))
])

transform = T.Compose([
    T.ToPILImage(),
    T.Resize((100, 100)),
    T.ToTensor(),
    T.Normalize((0.5,), (0.5,))
])

BATCH_SIZE = 32
train_dataset = LFWPairedDataset(pairs_train, labels_train, transform=transform_aug)
val_dataset   = LFWPairedDataset(pairs_val, labels_val, transform=transform)
test_dataset  = LFWPairedDataset(pairs_test, labels_test, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

### A Arquitetura da Rede Siamesa

Uma rede siamesa consiste em duas (ou mais) sub-redes idênticas que compartilham pesos (parâmetros). O objetivo desta sub-rede, que chamaremos de `BaseNetwork`, é extrair um vetor de características (embedding) da imagem de entrada.

#### Rede Base (Feature Extractor)

A `BaseNetwork` será uma Rede Neural Convolucional (CNN) relativamente simples. Ela processará a imagem de entrada e produzirá um vetor latente de dimensão fixa (ex: 128). A qualidade deste embedding é o que o modelo deve aprender: imagens da mesma pessoa devem resultar em embeddings próximos no espaço vetorial.

In [ ]:
class BaseNetwork(nn.Module):
    def __init__(self, embedding_dim=128):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 32, 5, padding=2), nn.BatchNorm2d(32), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 5, padding=2), nn.BatchNorm2d(64), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64, 96, 3, padding=1), nn.BatchNorm2d(96), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.fc = nn.Sequential(
            nn.Linear(96 * 12 * 12, 256), nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(256, embedding_dim)
        )

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return F.normalize(x, p=2, dim=1)

#### Rede Siamesa Completa

A `SiameseNetwork` propriamente dita instancia a `BaseNetwork` (uma única vez, garantindo o compartilhamento de pesos). Seu método `forward` aceita duas imagens de entrada, `input1` e `input2`, passa cada uma delas pela `BaseNetwork` e retorna os dois vetores de embedding resultantes.

In [ ]:
class SiameseNetwork(nn.Module):
    def __init__(self, embedding_dim=128):
        super().__init__()
        self.base_network = BaseNetwork(embedding_dim)

    def forward(self, input1, input2):
        output1 = self.base_network(input1)
        output2 = self.base_network(input2)
        return output1, output2

### Função de Custo: Contrastive Loss

Para treinar uma rede siamesa, não utilizamos funções de custo de classificação tradicionais (como Cross-Entropy). Em vez disso, usamos uma função de custo baseada em distância, como a **Contrastive Loss**.

O objetivo desta função é:
1.  Se o par for da mesma classe (label $Y=1$), a distância Euclidiana $D_W$ entre seus embeddings ($E(X_1)$ e $E(X_2)$) deve ser minimizada.
2.  Se o par for de classes diferentes (label $Y=0$), a distância $D_W$ deve ser maximizada, mas apenas até ultrapassar uma certa **margem** ($m$). Se a distância já for maior que a margem, a perda é zero (o modelo já separou bem o par).

A distância Euclidiana é definida como:
$$D_W = || E(X_1) - E(X_2) ||_2$$

A Contrastive Loss é formulada como:
$$L(W, (Y, X_1, X_2)) = Y \cdot \frac{1}{2} (D_W)^2 + (1 - Y) \cdot \frac{1}{2} \{ \max(0, m - D_W) \}^2$$

In [ ]:
class ContrastiveLoss(nn.Module):
    def __init__(self, margin=1.0):
        super().__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        # Distância euclidiana entre embeddings normalizados
        dist = F.pairwise_distance(output1, output2)

        # Perda para pares positivos (mesma classe)
        pos_loss = label * torch.pow(dist, 2)

        # Perda para pares negativos (classes diferentes)
        neg_loss = (1 - label) * torch.pow(torch.clamp(self.margin - dist, min=0.0), 2)

        # Média das perdas
        loss = torch.mean(pos_loss + neg_loss)
        return loss

### Configuração do Treinamento

Definimos os hiperparâmetros, instanciamos o modelo, a função de custo (Contrastive Loss) e o otimizador (Adam). Também configuramos o dispositivo de hardware (GPU, se disponível, ou CPU).

In [ ]:
# Configurações
MARGIN = 1.4

# Instanciação
model = SiameseNetwork(embedding_dim=256).to(device)
criterion = ContrastiveLoss(margin=MARGIN)
optimizer = optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-5)

### Loop de Treinamento e Validação

Iteramos sobre as épocas de treinamento. Em cada época, primeiro executamos a fase de treinamento (`model.train()`), onde calculamos a perda, realizamos a retropropagação (backpropagation) e atualizamos os pesos. Em seguida, executamos a fase de validação (`model.eval()`), onde apenas calculamos a perda nos dados de validação (sem calcular gradientes) para monitorar a generalização do modelo e detectar overfitting.

In [ ]:
NUM_EPOCHS = 25
train_losses = []
val_losses = []

for epoch in range(NUM_EPOCHS):
    model.train()
    running_train_loss = 0.0
    
    for i, (img1, img2, label) in enumerate(train_loader):
        img1, img2, label = img1.to(device), img2.to(device), label.to(device)

        optimizer.zero_grad()
        output1, output2 = model(img1, img2)
        loss = criterion(output1, output2, label)
        loss.backward()
        optimizer.step()
        running_train_loss += loss.item()
        
    epoch_train_loss = running_train_loss / len(train_loader)
    train_losses.append(epoch_train_loss)

    model.eval()
    running_val_loss = 0.0

    with torch.no_grad():
        for img1, img2, label in val_loader:
            img1, img2, label = img1.to(device), img2.to(device), label.to(device)

            output1, output2 = model(img1, img2)
            loss = criterion(output1, output2, label)
            running_val_loss += loss.item()
            
    epoch_val_loss = running_val_loss / len(val_loader)
    val_losses.append(epoch_val_loss)
    
    print(f"Epoch {epoch+1}/{NUM_EPOCHS} | Train Loss: {epoch_train_loss:.4f} | Val Loss: {epoch_val_loss:.4f}")

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Contrastive Loss')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc

def compute_distances_and_labels(model, dataloader, device, mode="contrastive"):
    model.eval()
    dists, labels = [], []

    with torch.no_grad():
        for batch in dataloader:
            if mode == "contrastive":
                x1, x2, y = [x.to(device) for x in batch]
                e1, e2 = model(x1, x2)
                dist = F.pairwise_distance(e1, e2)
                dists.extend(dist.cpu().numpy())
                labels.extend(y.cpu().numpy())

            elif mode == "triplet":
                a, p, n = [x.to(device) for x in batch]
                ea, ep, en = model(a), model(p), model(n)
                d_pos = F.pairwise_distance(ea, ep)
                d_neg = F.pairwise_distance(ea, en)
                dists.extend(d_pos.cpu().numpy())
                labels.extend(np.ones(len(d_pos)))
                dists.extend(d_neg.cpu().numpy())
                labels.extend(np.zeros(len(d_neg)))

    return np.array(dists), np.array(labels)


def evaluate_model(model, dataloader, device, mode="contrastive", title="ROC Curve"):
    dists, labels = compute_distances_and_labels(model, dataloader, device, mode)

    fpr, tpr, thr = roc_curve(labels, -dists)
    auc_val = auc(fpr, tpr)
    best = np.argmax(tpr - fpr)
    best_thr = -thr[best]
    preds = (dists < best_thr).astype(int)
    acc = (preds == labels).mean()

    plt.figure(figsize=(6,6))
    plt.plot(fpr, tpr, label=f"AUC={auc_val:.3f}")
    plt.scatter(fpr[best], tpr[best], c='red', label=f"thr={best_thr:.3f}")
    plt.plot([0,1],[0,1],'--',color='gray')
    plt.xlabel("FPR")
    plt.ylabel("TPR")
    plt.title(title)
    plt.legend()
    plt.grid()
    plt.show()

    print(f"AUC: {auc_val:.3f}")
    print(f"Melhor threshold: {best_thr:.3f}")
    print(f"Acurácia: {acc*100:.2f}%")
    print(f"Sensibilidade (TPR): {tpr[best]:.3f}")
    print(f"Especificidade (1 - FPR): {1-fpr[best]:.3f}")

    return auc_val, acc, best_thr


def test_model(model, dataloader, device, threshold, mode="contrastive"):
    dists, labels = compute_distances_and_labels(model, dataloader, device, mode)
    preds = (dists < threshold).astype(int)
    acc = (preds == labels).mean()
    print(f"Acurácia no teste (thr={threshold:.3f}): {acc*100:.2f}%")
    return acc

In [ ]:
auc_val, acc_val, best_thr = evaluate_model(model, val_loader, device, mode="contrastive")
acc_test = test_model(model, test_loader, device, best_thr, mode="contrastive")

### Inferência no Conjunto de Teste

Para realizar a inferência, passamos pares de imagens do conjunto de teste pela rede (em modo `model.eval()`). Calculamos a distância Euclidiana entre os embeddings de saída. Um limiar (threshold) é usado para tomar a decisão: se a distância for menor que o limiar, o modelo prevê que são a mesma pessoa; caso contrário, prevê que são pessoas diferentes. Um limiar comum para iniciar é `margin / 2`, mas o ideal é que ele seja sintonizado usando o conjunto de validação.

In [ ]:
def get_samples_to_plot(dataloader, mode="contrastive", num_pos=5, num_neg=5):
    samples = []
    for batch in dataloader:
        if mode == "contrastive":
            img1, img2, labels = batch
            for i in range(len(labels)):
                label = labels[i].item()
                if label == 1.0 and len([s for s in samples if s[2] == 1.0]) < num_pos:
                    samples.append((img1[i].cpu(), img2[i].cpu(), 1.0))
                elif label == 0.0 and len([s for s in samples if s[2] == 0.0]) < num_neg:
                    samples.append((img1[i].cpu(), img2[i].cpu(), 0.0))
            if len(samples) >= num_pos + num_neg:
                break

        elif mode == "triplet":
            anchor, positive, negative = batch
            for i in range(min(num_pos, len(anchor))):
                samples.append((anchor[i].cpu(), positive[i].cpu(), negative[i].cpu()))
            if len(samples) >= num_pos:
                break

    print(f"Total de samples coletados: {len(samples)}")
    return samples

In [ ]:
def imshow(img, text=None):
    img = img * 0.5 + 0.5  # desnormaliza
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)).squeeze(), cmap='gray')
    if text:
        plt.title(text)
    plt.axis('off')

def show_inference_examples(model, samples, device, threshold, mode="contrastive", max_examples=10):
    model.eval()

    for i, sample in enumerate(samples[:max_examples]):
        if mode == "contrastive":
            img1, img2, label = sample
            with torch.no_grad():
                d = F.pairwise_distance(*model(img1.unsqueeze(0).to(device),
                                               img2.unsqueeze(0).to(device)))
                distance = d.item()
            pred = "Same" if distance < threshold else "Different"
            truth = "Same" if label == 1.0 else "Different"

            print(f"\nExemplo {i+1}")
            print(f"Distância: {distance:.4f}")
            print(f"Predição: {pred} (Limiar: {threshold:.2f})")
            print(f"Verdadeiro: {truth}")

            fig, axes = plt.subplots(1, 2, figsize=(5, 2.5))
            for ax, img in zip(axes, [img1, img2]):
                plt.sca(ax)
                imshow(img)
            plt.show()

        elif mode == "triplet":
            anchor, positive, negative = sample
            with torch.no_grad():
                ea, ep, en = model(anchor.unsqueeze(0).to(device)), \
                             model(positive.unsqueeze(0).to(device)), \
                             model(negative.unsqueeze(0).to(device))
                d_pos = F.pairwise_distance(ea, ep).item()
                d_neg = F.pairwise_distance(ea, en).item()

            print(f"\nExemplo {i+1}")
            print(f"Distância (âncora-positivo): {d_pos:.4f}")
            print(f"Distância (âncora-negativo): {d_neg:.4f}")
            print(f"Decisão: {'Correto' if d_pos < d_neg else 'Incorreto'} (Limiar: {threshold:.2f})")

            fig, axes = plt.subplots(1, 3, figsize=(7, 2.5))
            for ax, img, title in zip(axes, [anchor, positive, negative], ["Âncora", "Positivo", "Negativo"]):
                plt.sca(ax)
                imshow(img, text=title)
            plt.show()

In [ ]:
samples_to_plot = get_samples_to_plot(test_loader, mode="contrastive", num_pos=5, num_neg=5)
show_inference_examples(model, samples_to_plot, device, best_thr, mode="contrastive")

# Triplet Loss

Embora a Contrastive Loss funcione bem com pares de imagens, outra abordagem predominante no aprendizado de métricas (metric learning) é a **Triplet Loss** (Perda Tripla). Esta função de custo não opera em pares (positivo/negativo), mas em *tripletos* de amostras.

Um tripleto consiste em:
1.  **Âncora ($A$):** Uma imagem de referência (ex: um retrato de uma pessoa).
2.  **Positivo ($P$):** Uma imagem diferente, mas da *mesma classe* que a âncora (ex: outro retrato da mesma pessoa).
3.  **Negativo ($N$):** Uma imagem de uma classe *diferente* da âncora (ex: um retrato de uma pessoa diferente).

O objetivo da Triplet Loss é modificar o espaço de embedding de forma que a distância entre a âncora e o positivo seja menor do que a distância entre a âncora e o negativo.

### Mineração de Tripletos (Triplet Mining)

Um desafio central na utilização da Triplet Loss é a seleção de tripletos. Se selecionarmos tripletos aleatoriamente, muitos deles serão "fáceis" (onde $D(A, N)$ já é muito maior que $D(A, P) + m$), resultando em uma perda nula e nenhum aprendizado.

Para um treinamento eficaz, é crucial empregar estratégias de "mineração de tripletos" (Triplet Mining):
1.  **Hard Negative/Positive Mining:** Selecionar os exemplos positivos mais distantes ($D(A, P)$) e os exemplos negativos mais próximos ($D(A, N)$) dentro de um lote (batch).
2.  **Semi-Hard Negative Mining:** Selecionar negativos que violam a margem ($D(A, P) < D(A, N) < D(A, P) + m$). Esta é uma abordagem muito comum, pois evita negativos excessivamente difíceis (colapsando o modelo) e ignora os negativos fáceis.

In [ ]:
import random

class LFWTripletDataset(Dataset):
    def __init__(self, pairs, labels, transform=None):
        self.transform = transform
        self.same_pairs = [p for p, l in zip(pairs, labels) if l == 1]
        self.diff_imgs = [img for p, l in zip(pairs, labels) if l == 0 for img in p]

    def __len__(self):
        return len(self.same_pairs)

    def __getitem__(self, idx):
        # Âncora e positivo vêm de um par positivo
        anchor, positive = self.same_pairs[idx]
        # Negativo vem de uma imagem de outro par (aleatório)
        negative = random.choice(self.diff_imgs)

        # Expande o canal (grayscale → [H,W,1])
        anchor = np.expand_dims(anchor, -1)
        positive = np.expand_dims(positive, -1)
        negative = np.expand_dims(negative, -1)

        # Aplica transformações
        if self.transform:
            anchor = self.transform(anchor)
            positive = self.transform(positive)
            negative = self.transform(negative)

        return anchor, positive, negative

In [ ]:
BATCH_SIZE = 64
train_triplet_dataset = LFWTripletDataset(pairs_train, labels_train, transform=transform_aug)
val_triplet_dataset   = LFWTripletDataset(pairs_val, labels_val, transform=transform)
test_triplet_dataset  = LFWTripletDataset(pairs_test, labels_test, transform=transform)

train_triplet_loader = DataLoader(train_triplet_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_triplet_loader   = DataLoader(val_triplet_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_triplet_loader  = DataLoader(test_triplet_dataset, batch_size=BATCH_SIZE, shuffle=False)

### Implementação da Triplet Loss

O objetivo da Triplet Loss é garantir que a distância entre a Âncora ($A$) e o Positivo ($P$) seja menor que a distância entre a Âncora ($A$) e o Negativo ($N$), respeitando uma margem ($m$).

A distância Euclidiana (L2) entre dois vetores $X_1$ e $X_2$ é:
$$D(X_1, X_2) = || X_1 - X_2 ||_2$$

A função de custo para um único tripleto é:
$$L(A, P, N) = \max(0, D(A, P) - D(A, N) + m)$$

Para implementar isso em PyTorch, seguiremos três passos dentro da classe:
1.  **Cálculo das Distâncias:** Usaremos `F.pairwise_distance` para calcular $D(A, P)$ e $D(A, N)$. Esta função calcula eficientemente a distância L2 (quando `p=2`) entre cada par de vetores em um lote.
2.  **Aplicação da Fórmula:** Implementaremos a lógica $D(A, P) - D(A, N) + m$.
3.  **Aplicação do Max(0, ...):** Usaremos `F.relu` para aplicar a função $\max(0, \cdot)$, que zera a perda para tripletos que já satisfazem a condição da margem.
4.  **Média do Lote:** Por fim, calculamos a média (`torch.mean`) da perda sobre todos os tripletos no lote.

In [ ]:
class TripletLoss(nn.Module):
    def __init__(self, margin=1.0):
        super().__init__()
        self.margin = margin

    def forward(self, anchor, positive, negative):
        dist_pos = F.pairwise_distance(anchor, positive)
        dist_neg = F.pairwise_distance(anchor, negative)
        loss = F.relu(dist_pos.pow(2) - dist_neg.pow(2) + self.margin)
        return loss.mean()

In [ ]:
model_triplet = BaseNetwork(embedding_dim=256).to(device)
criterion = TripletLoss(margin=1.0)
optimizer = optim.AdamW(model_triplet.parameters(), lr=1e-4)

In [ ]:
NUM_EPOCHS = 25
train_losses, val_losses = [], []

for epoch in range(NUM_EPOCHS):
    model_triplet.train()
    running_loss = 0
    for a, p, n in train_triplet_loader:
        a, p, n = a.to(device), p.to(device), n.to(device)
        optimizer.zero_grad()
        emb_a, emb_p, emb_n = model_triplet(a), model_triplet(p), model_triplet(n)
        loss = criterion(emb_a, emb_p, emb_n)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    train_loss = running_loss / len(train_loader)
    train_losses.append(train_loss)

    model_triplet.eval()
    val_loss = 0
    with torch.no_grad():
        for a, p, n in val_triplet_loader:
            a, p, n = a.to(device), p.to(device), n.to(device)
            emb_a, emb_p, emb_n = model_triplet(a), model_triplet(p), model_triplet(n)
            loss = criterion(emb_a, emb_p, emb_n)
            val_loss += loss.item()
    val_loss /= len(val_triplet_loader)
    val_losses.append(val_loss)

    print(f"Epoch {epoch+1}/{NUM_EPOCHS} | Train: {train_loss:.4f} | Val: {val_loss:.4f}")

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(train_losses, label="Train")
plt.plot(val_losses, label="Val")
plt.xlabel("Épocas")
plt.ylabel("Triplet Loss")
plt.legend()
plt.grid()
plt.show()

In [ ]:
auc_val, acc_val, best_triplet_thr = evaluate_model(model_triplet, val_triplet_loader, device, mode="triplet")
acc_test = test_model(model_triplet, test_triplet_loader, device, best_thr, mode="triplet")

In [ ]:
samples_to_plot = get_samples_to_plot(test_triplet_loader, mode="triplet", num_pos=2, num_neg=2)
show_inference_examples(model_triplet, samples_to_plot, device, best_triplet_thr, mode="triplet")